# Resume Scoring System 

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

# Set wide layout
st.set_page_config(layout="wide")

# Widen the Streamlit canvas
st.markdown("""
    
""", unsafe_allow_html=True)

# Connect to Snowflake
session = st.connection('snowflake').session()

# SQL query to fetch candidate data from Snowflake
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE 
    FROM CANDIDATE_PROFILES
    """
    return session.sql(query).to_pandas()

user_data = fetch_data()

# Job Roles
job_roles = pd.DataFrame({
    'job_role': ['Data Analyst', 'BI Consultant', 'Data Scientist', 'Data Engineer'],
    'base_weight': [70, 80, 85, 90]
})

# Job Levels
job_levels = pd.DataFrame({
    'job_level': ['Junior', 'Mid-level', 'Senior', 'Expert'],
    'level_weight': [0.5, 1, 1.5, 2]
})

# Skill Weights
skills = pd.DataFrame({
    'skill': [
        'AWS', 'Apache Airflow', 'Apache Kafka', 'Apache Spark', 'DAX', 'Excel', 'Hadoop', 
        'NLP', 'Power BI', 'Python', 'R', 'SQL', 'Tableau', 'TensorFlow'
    ],
    'Data Analyst': [5, 0, 0, 0, 20, 15, 0, 0, 25, 15, 0, 20, 20, 0],
    'BI Consultant': [0, 10, 5, 5, 20, 10, 10, 5, 25, 15, 10, 15, 25, 10],
    'Data Scientist': [10, 15, 15, 10, 25, 15, 10, 10, 15, 20, 20, 20, 15, 30],
    'Data Engineer': [20, 20, 25, 25, 10, 10, 20, 15, 10, 20, 10, 25, 5, 15]
})

# Experience weights
experience = pd.DataFrame({
    'years_of_experience': [1, 2, 3, 4, 5, 6, 7],
    'experience_weight': [10, 15, 20, 25, 30, 35, 40]
})

# Helper Functions
def get_weight(df, column, value, weight_col):
    row = df[df[column] == value]
    return row[weight_col].values[0] if not row.empty else 0

def get_role_specific_skill_weight(skill, job_role):
    row = skills[skills['skill'] == skill]
    return row[job_role].values[0] if not row.empty else 0

def calculate_weight(job_role, job_level, selected_skills, years_exp):
    role_weight = get_weight(job_roles, 'job_role', job_role, 'base_weight')
    level_weight = get_weight(job_levels, 'job_level', job_level, 'level_weight')
    skill_weight = sum([get_role_specific_skill_weight(s, job_role) for s in selected_skills])
    exp_weight = get_weight(experience, 'years_of_experience', years_exp, 'experience_weight')
    total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
    return round(total, 2)

def candidate_score(row, selected_job, selected_level, selected_skills, selected_exp):
    role_weight = get_weight(job_roles, 'job_role', selected_job, 'base_weight')
    candidate_level = row['JOB_LEVEL']
    candidate_exp = row['YEARS_OF_EXPERIENCE']
    level_weight = get_weight(job_levels, 'job_level', candidate_level, 'level_weight')
    candidate_skills = set(map(str.strip, row['TECHNICAL_SKILLS'].split(',')))
    matched_skills = set(selected_skills).intersection(candidate_skills)
    skill_weight = sum([get_role_specific_skill_weight(s, selected_job) for s in matched_skills])
    exp_weight = get_weight(experience, 'years_of_experience', candidate_exp, 'experience_weight')
    total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
    return round(total)

# --- Streamlit App ---
st.title("📊 ATS Resume Ranking System")
min_exp = int(experience['years_of_experience'].min())
max_exp = int(experience['years_of_experience'].max())

# Inputs
selected_job = st.sidebar.multiselect("Select Job Role", job_roles['job_role'])
#all_levels = job_levels['job_level'].tolist()
selected_levels = st.sidebar.multiselect("Select Job Level(s)", job_levels['job_level'])
selected_skills = st.sidebar.multiselect("Select Required Skills", skills['skill'])
selected_exp = st.sidebar.slider("Select Minimum Years of Experience", min_value=min_exp, max_value=max_exp, value=min_exp)
top_n = st.sidebar.selectbox("Select number of top candidates to display", ['All', 1, 2, 3, 4, 5])

# Filter Candidates
filtered_data = user_data.copy()

if selected_job:
    filtered_data = filtered_data[filtered_data['JOB_TITLE'].isin(selected_job)]

# Filter by selected job levels
if selected_levels:
    filtered_data = filtered_data[filtered_data['JOB_LEVEL'].isin(selected_levels)]

# Filter by skills
filtered_data = filtered_data[
    filtered_data['TECHNICAL_SKILLS'].apply(
        lambda s: set(selected_skills).issubset(set(map(str.strip, s.split(','))))
    )
]

# Filter by experience
filtered_data = filtered_data[
    filtered_data['YEARS_OF_EXPERIENCE'] >= selected_exp
]

# Check if filtered data is empty
if filtered_data.empty:
    st.warning("No candidates match the selected criteria.")
else:
    # Use default level for scoring if 'All' levels were selected
    selected_level_for_score = selected_levels[0] if selected_levels else 'Junior'
    selected_job_for_score = selected_job[0] if selected_job else 'Data Analyst'

    baseline_weight = calculate_weight(selected_job_for_score, selected_level_for_score, selected_skills, selected_exp)
    st.markdown(f"### Baseline Competency Weight: `{baseline_weight}`")

    # Score all candidates using selected inputs
    filtered_data['competency_weight'] = filtered_data.apply(
        lambda row: candidate_score(row, selected_job_for_score, selected_level_for_score, selected_skills, selected_exp),
        axis=1
    )

    filtered_data['status'] = filtered_data['competency_weight'].apply(
        lambda x: "✓" if x >= baseline_weight else "✗"
    )

    # Sort and select top candidates
    ranked_users = filtered_data.sort_values(by='competency_weight', ascending=False)
    if top_n != 'All':
        ranked_users = ranked_users.head(int(top_n))

    # Styling
    def highlight_status(val):
        color = 'green' if val == '✓' else 'red'
        return f'color: {color}; font-weight: bold;'

    styled_df = ranked_users[[
        'FULL_NAME', 'PHONE_NUMBER', 'EMAIL', 'LOCATION', 'JOB_TITLE', 'JOB_LEVEL', 
        'TECHNICAL_SKILLS', 'YEARS_OF_EXPERIENCE', 'competency_weight', 'status'
    ]].style.applymap(highlight_status, subset=['status'])

    st.subheader("🏅 Ranked Candidates")
    st.dataframe(styled_df, use_container_width=True)


    #Resume Card Profile 
    original_df = styled_df.data  # Get the original DataFrame
    
    for idx, row in original_df.iterrows():

        #getting the skills
        skill_list = [s.strip() for s in row['TECHNICAL_SKILLS'].split(',') if s.strip()]
        
        #define color for each titles and job levels
        job_title_colors = {"Data Scientist": "#b3e5fc", "Data Analyst": "#ffccbc", "BI Consultant": "#c8e6c9", "Data Scientist ": "#f0f4c3"}
        job_level_colors = {"Junior": "#ffe082", "Mid-level": "#ffeb3b", "Senior": "#f57c00"}

        #default color 
        job_title_color = job_title_colors.get(row['JOB_TITLE'], "#e0f7fa")
        job_level_color = job_level_colors.get(row['JOB_LEVEL'], "#fce4ec")

        #set border between each resume
        with st.container():
            st.markdown(
                f"""
                <div style='
                    border: 1px solid #ddd;
                    border-radius: 8px;
                    padding: 5px 15px;
                    margin-bottom: 10px;
                    background-color: #fff;
                '>
                """,
                unsafe_allow_html=True
            )

            #name, experience, job title, job level 
            inner_cols = st.columns([3, 3, 2, .5])
            with inner_cols[0]:
                st.markdown(
                    """
                    <div style="margin: 20px;">
                        <h3>{}</h3>
                        <p><strong>Experience:</strong> {} years</p>
                        <div style="margin-bottom: 15px;">
                            <span style="background-color:{}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px">
                                {}
                            </span>
                            <span style="background-color:{}; color:#880e4f; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px">
                                {}
                            </span>
                        </div>
                    </div>
                    """.format(
                        row['FULL_NAME'], 
                        row['YEARS_OF_EXPERIENCE'], 
                        job_title_color, 
                        row['JOB_TITLE'], 
                        job_level_color, 
                        row['JOB_LEVEL']
                    ),
                    unsafe_allow_html=True  
                )


            #technical skills 
            with inner_cols[1]:
                st.markdown(f"**Skills:**")
                skill_colors = ["#a5d6a7", "#c5cae9", "#ffcc80", "#ffab91", "#80deea", "#f48fb1"]
                skill_badges = " ".join([
                    f"<span style='background-color:{skill_colors[i % len(skill_colors)]}; color:#2e7d32; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>{s}</span>"
                    for i, s in enumerate(skill_list)
                ])
                st.markdown(f"<div style='margin-top:10px; display: flex; flex-wrap: wrap'>{skill_badges}</div>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)


            with inner_cols[2]:
                st.markdown(f"**Competency Weight:**")
                st.markdown(
                    f"""
                    <span style='
                        background-color:#d0f0c0; 
                        color:#2e7d32; 
                        padding:8px 16px; 
                        border-radius:12px; 
                        font-size:16px; 
                        font-weight:bold; 
                    '>
                        {row['competency_weight']}
                    </span>
                    """,
                    unsafe_allow_html=True
                )



            st.markdown("</div>", unsafe_allow_html=True)
